In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
import MeshCat as mc 

  Activating project at `c:\Users\zsqu4re\Desktop\OCRL\Optimal-Control-and-Reinforcement-Learning\HW3_S25-main`
    Updating registry at `C:\Users\zsqu4re\.julia\registries\General.toml`
   Installed GR_jll ──────────────────── v0.73.13+0
   Installed ConcurrentUtilities ─────── v2.5.0
   Installed StaticArrays ────────────── v1.9.13
   Installed MutableArithmetics ──────── v1.6.4
   Installed ECOS ────────────────────── v1.1.3
   Installed FileIO ──────────────────── v1.17.0
   Installed Cairo_jll ───────────────── v1.18.4+0
   Installed OpenBLAS32_jll ──────────── v0.3.24+0
   Installed Pango_jll ───────────────── v1.56.1+0
   Installed MUMPS_seq_jll ───────────── v500.700.301+0
   Installed FiniteDiff ──────────────── v2.27.0
   Installed Hwloc_jll ───────────────── v2.12.0+0
   Installed NBInclude ───────────────── v2.4.0
   Installed ColorSchemes ────────────── v3.29.0
   Installed GR ──────────────────────── v0.73.13
   Installed Ipopt_jll ───────────────── v300.1400.1700+0
   In

In [2]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","cartpole_animation.jl"))

animate_cartpole (generic function with 1 method)

**NOTE: This question will have long outputs for each cell, remember you can use `cell -> all output -> toggle scrolling` to better see it all**

## Q1: Direct Collocation (DIRCOL) for a Cart Pole (30 pts)

We are now going to start working with the NonLinear Program (NLP) Solver IPOPT to solve some trajectory optimization problems. First we will demonstrate how this works for simple optimization problems (not trajectory optimization). The interface that we have setup for IPOPT is the following:

$$ \begin{align} \min_{x} \quad & \ell(x) & \text{cost function}\\ 
 \text{st} \quad & c_{eq}(x) = 0 & \text{equality constraint}\\
 & c_L \leq c_{ineq}(x) \leq c_U & \text{inequality constraint}\\
 & x_L \leq x \leq x_U & \text{primal bound constraint}
 \end{align}$$
 
 where $\ell(x)$ is our objective function, $c_{eq}(x) = 0$ is our equality constraint, $c_L \leq c_{ineq}(x) \leq c_U$ is our bound inequality constraint, and $x_L \leq x \leq x_U $ is a bound constraint on our primal variable $x$. 
<!-- $$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ \frac{1}{2} x_i^TQx_i + \frac{1}{2} u_i^TRu_i \bigg] + \frac{1}{2}x_N^TQ_fx_N\\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_{i+1} = A x_i + Bu_i \quad \text{for } i = 1,2,\ldots,N-1 
 \end{align}$$ -->

## Part A: Solve an LP with IPOPT (5 pts)
To demonstrate this, we are going to ask you to solve a simple Linear Program (LP):
 
 $$ \begin{align} \min_{x} \quad & q^Tx\\ 
 \text{st} \quad & Ax = b \\
 & Gx \leq h
 \end{align}$$
 
 Your job will be to transform this problem into the form shown above and solve it with IPOPT.  To help you interface with IPOPT, we have created a function `fmincon` for you. Below is the docstring for this function that details all of the inputs. 

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [8]:
@testset "solve LP with IPOPT" begin 
        
    LP = jldopen(joinpath(@__DIR__,"utils","random_LP.jld2"))
    
    params = (q = LP["q"], A = LP["A"], b = LP["b"], G = LP["G"], h = LP["h"])
    
    # return a scalar 
    function cost(params, x)::Real
        # TODO: create cost function with params and x 
        return params.q' * x
        
    end
    
    # return a vector 
    function equality_constraint(params, x)::Vector
        # TODO: create equality constraint function with params and x 
        return params.A * x - params.b
        
    end
    
    # return a vector
    function inequality_constraint(params, x)::Vector
        # TODO: create inequality constraint function with params and x 
        return params.G * x - params.h
        
    end

    # TODO: primal bounds 
    # you may use Inf, like Inf*ones(10) for a vector of positive infinity 
    x_l = -Inf * ones(20)
    x_u = Inf * ones(20)
    
    # TODO: inequality constraint bounds 
    c_l = -Inf * ones(size(params.G, 1))
    c_u = zeros(size(params.G, 1))

    
    # initial guess 
    x0= randn(20)
    
    diff_type = :auto   # use ForwardDiff.jl
#     diff_type = :finite # use FiniteDiff.jl
    
    x = fmincon(cost, equality_constraint, inequality_constraint,
                x_l, x_u, c_l, c_u, x0, params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true);
    
    
    @test isapprox(x, [-0.44289, 0, 0, 0.19214, 0, 0, -0.109095,
                       -0.43221, 0, 0, 0.44289, 0, 0, 0.192142,
                       0, 0, 0.10909, 0.432219, 0, 0], atol = 1e-3)

end

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       80
Number of nonzeros in inequality constraint Jacobian.:      400
Number of nonzeros in Lagrangian Hessian..

Test.DefaultTestSet("solve LP with IPOPT", Any[], 1, false, false, true, 1.742311905124e9, 1.742311909073e9, false, "c:\\Users\\zsqu4re\\Desktop\\OCRL\\Optimal-Control-and-Reinforcement-Learning\\HW3_S25-main\\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sZmlsZQ==.jl")

## Part B: Cart Pole Swingup (20 pts)

We are now going to solve for a cartpole swingup. The state for the cartpole is the following: 

$$ x = [p,\theta,\dot{p},\dot{\theta}]^T $$ 

Where $p$ and $\theta$ can be seen in the graphic `cartpole.png`.

<div>
<img src="attachment:cartpole.png" width="300"/>
</div>

where we start with the pole in the down position ($\theta = 0$), and we want to use the horizontal force on the cart to drive the pole to the up position ($\theta = \pi$).

$$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ \frac{1}{2} (x_i - x_{goal})^TQ(x_i - x_{goal}) + \frac{1}{2} u_i^TRu_i \bigg] + \frac{1}{2}(x_N - x_{goal})^TQ_f(x_N - x_{goal})\\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_N = x_{goal} \\ 
 & f_{hs}(x_i,x_{i+1},u_i,dt) = 0 \quad \text{for } i = 1,2,\ldots,N-1 \\
 & -10 \leq u_i \leq 10 \quad \text{for } i = 1,2,\ldots,N-1 
 \end{align}$$
 
 Where $x_{IC} = [0,0,0,0]$, and $x_{goal} = [0, \pi, 0, 0]$, and $f_{hs}(x_i,x_{i+1},u_i)$ is the implicit integrator residual for Hermite Simpson (see HW1Q1 to refresh on this). Note that while Zac used a first order hold (FOH) on the controls in class (meaning we linearly interpolate controls between time steps), we are using a zero-order hold (ZOH) in this assignment. This means that each control $u_i$ is held constant for the entirety of the timestep. 

In [ ]:
# cartpole 
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole ODE, parametrized by params. 

    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    xdot = [qd;qdd]
    return xdot 

end
function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
    xmid = 0.5 * (x1 + x2) + (dt / 8) * (dynamics(params, x1, u) - dynamics(params, x2, u))
    f1 = dynamics(params, x1, u)
    fmid = dynamics(params, xmid, u)
    f2 = dynamics(params, x2, u)
    residual = x2 - x1 - (dt / 6) * (f1 + 4 * fmid + f2)
    return residual
end

hermite_simpson (generic function with 1 method)

To solve this problem with IPOPT and `fmincon`, we are going to concatenate all of our $x$'s and $u$'s into one vector:

$$ Z = \begin{bmatrix}x_1 \\ u_1 \\ x_2 \\ u_2 \\ \vdots \\ x_{N-1} \\ u_{N-1} \\ x_N \end{bmatrix} \in \mathbb{R}^{N \cdot nx + (N-1)\cdot nu} $$ 

where $x \in \mathbb{R}^{nx}$ and $u \in \mathbb{R}^{nu}$. Below we will provide useful indexing guide in `create_idx` to help you deal with $Z$.

It is also worth noting that while there are inequality constraints present ($-10 \leq u_i \leq 10$), we do not need a specific `inequality_constraints` function as an input to `fmincon` since these are just bounds on the primal ($Z$) variable. You should use primal bounds in `fmincon` to capture these constraints.  

In [6]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    
    # TODO: input cartpole LQR cost 
    
    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 0 

    end
    
    # dont forget terminal cost 
    
    return J 
end

function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    
    # TODO: create dynamics constraints using hermite simpson 
    
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        c[idx.c[i]] = zeros(4)
    end
    return c 
end

function cartpole_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    # TODO: return all of the equality constraints 
    
    
    return zeros(10) # 10 is an arbitrary number 
end

function solve_cartpole_swingup(;verbose=true)
    
    # problem size 
    nx = 4 
    nu = 1 
    dt = 0.05
    tf = 2.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # LQR cost 
    Q = diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial and goal states 
    xic = [0, 0, 0, 0]
    xg = [0, pi, 0, 0]
    
    # load all useful things into params 
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx,mc = 1.0, mp = 0.2, l = 0.5)
    
    # TODO: primal bounds 
    x_l = 
    x_u =  
    

    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(0)
    c_u = zeros(0)
    function inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end
    
    # initial guess 
    z0 = 0.001*randn(idx.nz)
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    
    Z = fmincon(cartpole_cost,cartpole_equality_constraint,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, t_vec, params 
end

@testset "cartpole swingup" begin 
    
    X, U, t_vec = solve_cartpole_swingup(verbose=true)
    
    
    # --------------testing------------------
    @test isapprox(X[1],zeros(4), atol = 1e-4)
    @test isapprox(X[end], [0,pi,0,0], atol = 1e-4)
    Xm = hcat(X...)
    Um = hcat(U...)
    
    # --------------plotting-----------------
    display(plot(t_vec, Xm', label = ["p" "θ" "ṗ" "θ̇"], xlabel = "time (s)", title = "State Trajectory"))
    display(plot(t_vec[1:end-1],Um',label="",xlabel = "time (s)", ylabel = "u",title = "Controls"))
    
    # meshcat animation
    display(animate_cartpole(X, 0.05))
    
end

---------checking dimensions of everything----------
cartpole swingup: Error During Test at c:\Users\zsqu4re\Desktop\OCRL\Optimal-Control-and-Reinforcement-Learning\HW3_S25-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:126
  Got exception outside of a @test
  AssertionError: length(x0) == length(x_l) == length(x_u)
  Stacktrace:
    [1] fmincon(cost::typeof(cartpole_cost), equality_constraint::typeof(cartpole_equality_constraint), inequality_constraint::var"#inequality_constraint#38", x_l::Vector{Float64}, x_u::Vector{Float64}, c_l::Vector{Float64}, c_u::Vector{Float64}, x0::Vector{Float64}, params::@NamedTuple{Q::Matrix{Float64}, R::Matrix{Float64}, Qf::Matrix{Float64}, xic::Vector{Int64}, xg::Vector{Float64}, dt::Float64, N::Int64, idx::@NamedTuple{nx::Int64, nu::Int64, N::Int64, nz::Int64, nc::Int64, x::Vector{UnitRange{Int64}}, u::Vector{UnitRange{Int64}}, c::Vector{UnitRange{Int64}}}, mc::Float64, mp::Float64, l::Float64}, diff_type::Symbol; tol::Float64, 

TestSetException: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.

## Part C: Track DIRCOL Solution (5 pts)

Now, similar to HW2 Q2 Part C, we are taking a solution $X$ and $U$ from DIRCOL, and we are going to track the trajectory with TVLQR to account for model mismatch. While we used hermite-simpson integration for the dynamics constraints in DIRCOL, we are going to use RK4 for this simulation. Remember to clamp your control to be within the control bounds. 

In [7]:
function rk4(params::NamedTuple, x::Vector,u,dt::Float64)
    # vanilla RK4
    k1 = dt*dynamics(params, x, u)
    k2 = dt*dynamics(params, x + k1/2, u)
    k3 = dt*dynamics(params, x + k2/2, u)
    k4 = dt*dynamics(params, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

@testset "track cartpole swingup with TVLQR" begin 
    
    X_dircol, U_dircol, t_vec, params_dircol = solve_cartpole_swingup(verbose = false)
    
    N = length(X_dircol)
    dt = params_dircol.dt 
    x0 = X_dircol[1]
    
    # TODO: use TVLQR to generate K's 

    # use this for TVLQR tracking cost 
    Q = diagm([1,1,.05,.1])
    Qf = 100*Q
    R = 0.01*diagm(ones(1))

    
    # simulation
    Xsim = [zeros(4) for i = 1:N]
    Usim = [zeros(1) for i = 1:(N-1)]
    Xsim[1] = 1*x0 
    
    # here are the real parameters (different than the one we used for DIRCOL)
    # this model mismatch is what's going to require the TVLQR controller to track
    # the trajectory successfully. 
    params_real = (mc = 1.05, mp = 0.21, l = 0.48)
    
    # TODO: simulate closed loop system with both feedforward and feedback control
    # feedforward - the U_dircol controls that we solved for using dircol 
    # feedback - the TVLQR controls 
    for i = 1:(N-1)

        # add controller and simulation step 
        
    end
    
    
    # -----------------testing-----------------------
    xn = Xsim[N]
    @test norm(xn)>0
    @test 1e-6<norm(xn - X_dircol[end])<.8
    @test abs(abs(rad2deg(xn[2])) - 180) < 5 # within 5 degrees 
    @test maximum(norm.(Usim,Inf)) <= (10 + 1e-3)
    
    # -----------------plotting----------------------
    Xm = hcat(Xsim...)
    Xbarm = hcat(X_dircol...)
    plot(t_vec,Xbarm',ls=:dash, label = "",lc = [:red :green :blue :black])
    display(plot!(t_vec,Xm',title = "Cartpole TVLQR (-- is reference)",
                 xlabel = "time (s)", ylabel = "x",
                 label = ["p" "θ" "ṗ" "θ̇"],lc = [:red :green :blue :black]))
    
    Um = hcat(Usim...)
    Ubarm = hcat(U_dircol...)
    plot(t_vec[1:end-1],Ubarm',ls=:dash,lc = :blue, label = "")
    display(plot!(t_vec[1:end-1],Um',title = "Cartpole TVLQR (-- is reference)",
                 xlabel = "time (s)", ylabel = "u",lc = :blue, label = ""))
    
    # ----------------animate------------------------
    display(animate_cartpole(Xsim, 0.05))
    
end

track cartpole swingup with TVLQR: Error During Test at c:\Users\zsqu4re\Desktop\OCRL\Optimal-Control-and-Reinforcement-Learning\HW3_S25-main\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:10
  Got exception outside of a @test
  AssertionError: length(x0) == length(x_l) == length(x_u)
  Stacktrace:
    [1] fmincon(cost::typeof(cartpole_cost), equality_constraint::typeof(cartpole_equality_constraint), inequality_constraint::var"#inequality_constraint#38", x_l::Vector{Float64}, x_u::Vector{Float64}, c_l::Vector{Float64}, c_u::Vector{Float64}, x0::Vector{Float64}, params::@NamedTuple{Q::Matrix{Float64}, R::Matrix{Float64}, Qf::Matrix{Float64}, xic::Vector{Int64}, xg::Vector{Float64}, dt::Float64, N::Int64, idx::@NamedTuple{nx::Int64, nu::Int64, N::Int64, nz::Int64, nc::Int64, x::Vector{UnitRange{Int64}}, u::Vector{UnitRange{Int64}}, c::Vector{UnitRange{Int64}}}, mc::Float64, mp::Float64, l::Float64}, diff_type::Symbol; tol::Float64, c_tol::Float64, max_iters::Int64, ver

TestSetException: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.